# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e compramos e vendemos commodities:
- Soja, Milho, Trigo, Petróleo, etc.

Precisamos pegar na internet, de forma automática, a cotação de todas as commodites e ver se ela está abaixo do nosso preço ideal de compra. Se tiver, precisamos marcar como uma ação de compra para a equipe de operações.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=share_link

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
from selenium import webdriver

# Essa forma utilizando o manager, faz o processo de baixar o driver e colocar no local correto automaticamente
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Nova maneira de localizar o elemento na página
from selenium.webdriver.common.by import By

service = Service(ChromeDriverManager().install())
navigator = webdriver.Chrome(service=service)


In [2]:
# !pip install --upgrade selenium
# Atualizar a versão mais nova do Selenium

#!pip install webdriver-manager
# Instalar o webdriver_manager para não precisar de atualizações futuras, cada vez que o Chrome atualize


In [3]:
import pandas as pd

# importar arquivo xlsx e exibir a tabela
tabela = pd.read_excel("commodities.xlsx")
display(tabela)


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,NaN,NaN
1,Soja,163.59,NaN,NaN
2,Boi,282.20,NaN,NaN
3,Petróleo,424.37,NaN,NaN
4,Algodão,497.76,NaN,NaN
5,Açúcar,136.23,NaN,NaN
6,Café,1092.87,NaN,NaN
7,Ouro,321.77,NaN,NaN
8,Trigo,1549.11,NaN,NaN
9,Tilápia,9.05,NaN,NaN


In [4]:
# Loop for para percorrer todas as linhas do tabela.index
# Em seguida tratamento de dados nos nomes dos produtos para substituir os caracteres especiais que temos por;
# caracteres normais, para ficar igual ao site
# No link temos o "f" que é chamado de "fstring", ele permite colocar uma variável dentro do texto;
# A variável {produto.lower()} com o Lower, para deixar todas palavras minusculas
for linha in tabela.index:
	produto = tabela.loc[linha, "Produto"]
	produto = produto.replace("ã", "a").replace("ç", "c").replace("á", 	"a").replace("ó", "o").replace("ú", "u").replace("é", "e")
	link = f"https://www.melhorcambio.com/{produto.lower()}-hoje"
	print(link)
	
	# Abrir o navegador, selecionar o produto e retornar com o valor
	# Usar o By.XPATH para pegar a posição de um elemento numa página e o "value", para retornar seu valor
	# Em seguida tratar o valores para ajustar as casas decimais e o separador de milhar, para deixar no formato americano
	navigator.get(link)
	preco = navigator.find_element(By.XPATH,'//*[@id="comercial"]').get_attribute("value")
	preco = preco.replace(".", "").replace(",", ".")
	print(preco)
	
	# Pegar os valores do preço atual de cada produto já tratados como float e inserir na tabela
	tabela.loc[linha, "Preço Atual"] = float(preco)
display(tabela)


https://www.melhorcambio.com/milho-hoje
53.88
https://www.melhorcambio.com/soja-hoje
146.99
https://www.melhorcambio.com/boi-hoje
199.65
https://www.melhorcambio.com/petroleo-hoje
439.30
https://www.melhorcambio.com/algodao-hoje
408.42
https://www.melhorcambio.com/acucar-hoje
149.24
https://www.melhorcambio.com/cafe-hoje
818.18
https://www.melhorcambio.com/ouro-hoje
311.63
https://www.melhorcambio.com/trigo-hoje
0.00
https://www.melhorcambio.com/tilapia-hoje
9.68


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,53.88,NaN
1,Soja,163.59,146.99,NaN
2,Boi,282.20,199.65,NaN
3,Petróleo,424.37,439.30,NaN
4,Algodão,497.76,408.42,NaN
5,Açúcar,136.23,149.24,NaN
6,Café,1092.87,818.18,NaN
7,Ouro,321.77,311.63,NaN
8,Trigo,1549.11,0.00,NaN
9,Tilápia,9.05,9.68,NaN


In [5]:
# Fechar o navegador
# Fazer a comparação dos valores e retornar true para comprar ou false para não comprar
# Salvar a tabela atualizada, em um novo arquivo de excel
navigator.quit()
tabela["Comprar"] = tabela["Preço Ideal"] > tabela["Preço Atual"]
tabela.to_excel("commodities_atualizada.xlsx", index=False)

display(tabela)


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,53.88,True
1,Soja,163.59,146.99,True
2,Boi,282.20,199.65,True
3,Petróleo,424.37,439.30,False
4,Algodão,497.76,408.42,True
5,Açúcar,136.23,149.24,False
6,Café,1092.87,818.18,True
7,Ouro,321.77,311.63,True
8,Trigo,1549.11,0.00,True
9,Tilápia,9.05,9.68,False


### Tabela Concluída

##### Preço Atual está atualizado com o valor do dia

Nosso objetivo é fazer uma comparação entre o Preço Ideal e o Preço Atual para saber<br/>
se devemos ou não comprar esses produtos.<br/>
Se o Preço Atual for menor que o ideal, **"True"** comprar ou **"False"** não comprar.
